In [1]:
from dask.distributed import Client,LocalCluster
import xarray as xr
import dask
import glob
import geopandas as gpd
import rioxarray as rio
import shapely
import os
os.environ['GDAL_DATA'] = os.environ['CONDA_PREFIX'] + r'/Library/share/gdal'
os.environ['PROJ_LIB'] = os.environ['CONDA_PREFIX'] + r'/Library/share'

In [2]:
# data_dir=r'C://Users/kerrie/Documents/02_LocalData/nclimgrid_daily/'
# shpfile=r'C://Users/kerrie/Documents/02_LocalData/boundaries/study_area_bounding_box.shp'
# data_dir=r'D://data/nclimgrid_daily/'
# shpfile=r'D://data/boundaries/study_area_bounding_box.shp'
data_dir=r'E://data/nclimgrid_daily/'
shpfile=r'E://data/boundaries/study_area_bounding_box.shp'

In [3]:
# this may require adjustment on other computers

# kerrie laptop client (machine has 32GB RAM and 20 threads)
# nworkers=20  # set equal to number of threads you have

# kerrie desktop client (machine has 64GB RAM and 16 threads)
nworkers=16  # set equal to number of threads you have

cluster=LocalCluster(n_workers=nworkers,threads_per_worker=1)
client=Client(cluster) 
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 16,Total memory: 63.25 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56869,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 63.25 GiB
Comm: tcp://127.0.0.1:56970,Total threads: 1
Dashboard: http://127.0.0.1:56971/status,Memory: 3.95 GiB
Nanny: tcp://127.0.0.1:56872,


2024-10-10 10:09:33,591 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f6d82ddf4bc2f017548057013090d287 initialized by task ('rechunk-merge-rechunk-transfer-ea492051fb233754d87470f0c36cb974', 0, 0, 0, 76, 0, 0) executed on worker tcp://127.0.0.1:56941
2024-10-10 10:12:47,759 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f6d82ddf4bc2f017548057013090d287 deactivated due to stimulus 'task-finished-1728573167.7599497'
2024-10-10 10:13:42,437 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2231bf862268310fc784562b678522df initialized by task ('rechunk-merge-rechunk-transfer-222611e73acff243582f8d5b59cd64d4', 0, 0, 0, 10, 0, 0) executed on worker tcp://127.0.0.1:56956
2024-10-10 10:17:04,321 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2231bf862268310fc784562b678522df deactivated due to stimulus 'task-finished-1728573424.3214757'


## Data Cleaning

1) drop any variables we don't need
2) dim names are time, lat, lon
3) time is correct datetimes
4) lat is ascending
5) lon is ascending -180 to 180
6) variable names tmax,tmin,prcp
7) units of C for temperature
8) units of mm/day for precip
9) spatial subset
11) round all values to 2 decimal places
12) save data with smallest possible precision
13) write cleaned files, 1 file per variable


In [4]:
files=sorted(glob.glob(data_dir+'orig/ncdd*-grd-scaled.nc'))
len(files), files[0:3]

(884,
 ['E://data/nclimgrid_daily/orig\\ncdd-195101-grd-scaled.nc',
  'E://data/nclimgrid_daily/orig\\ncdd-195102-grd-scaled.nc',
  'E://data/nclimgrid_daily/orig\\ncdd-195103-grd-scaled.nc'])

In [5]:
# lazy dataset, all 800+ files
ds=xr.open_mfdataset(files,chunks='auto',lock=False)#.chunk({'time':-1,'lat':-1,'lon':25})
# ds=xr.open_mfdataset(files,chunks={'time':-1,'lat':-1,'lon':25},lock=False)

ds

<xarray.Dataset> Size: 711GB
Dimensions:  (time: 26907, lat: 596, lon: 1385)
Coordinates:
  * time     (time) datetime64[ns] 215kB 1951-01-01 1951-01-02 ... 2024-08-31
  * lat      (lat) float32 2kB 24.56 24.6 24.65 24.69 ... 49.27 49.31 49.35
  * lon      (lon) float32 6kB -124.7 -124.6 -124.6 ... -67.1 -67.06 -67.02
Data variables:
    tmax     (time, lat, lon) float64 178GB dask.array<chunksize=(20, 596, 1385), meta=np.ndarray>
    tmin     (time, lat, lon) float64 178GB dask.array<chunksize=(20, 596, 1385), meta=np.ndarray>
    prcp     (time, lat, lon) float64 178GB dask.array<chunksize=(20, 596, 1385), meta=np.ndarray>
    tavg     (time, lat, lon) float64 178GB dask.array<chunksize=(20, 596, 1385), meta=np.ndarray>
Attributes: (12/33)
    date_created:               2022-08-27 17:20:14.814647
    date_modified:              2022-08-27 17:20:14.815683
    Conventions:                CF-1.7, ACDD-1.3
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    title:                      nClimGrid-Daily, Gridded Fields
    source:                     GHCN-Daily, nClimGrid-Monthly
    ...                         ...
    time_coverage_start:        1951-01-01
    time_coverage_end:          1951-01-31
    program:                    Center for Weather and Climate/Climate Scienc...
    geospatial_lat_resolution:  0.04167 degrees
    geospatial_lon_resolution:  0.04167 degrees
    platform:                   station

In [6]:
# 1) drop data we don't need"
ds = ds.drop_vars('tavg')

steps 2-7 aren't necessary, data already looks good wrt to these items

In [7]:
# step 8 millimeter --> mm/day
ds.prcp.attrs['units']='mm/day'

In [8]:
# step 9 spatial subset
# clip data to a bounding box

# get clip object
box=gpd.read_file(shpfile)

# assign crs to netcdf data
ds.rio.write_crs("epsg:4326",inplace=True)
ds_clip=ds.rio.clip(box.geometry.apply(shapely.geometry.mapping),box.crs,drop=True,invert=False)
ds_clip

C:\Users\kerrie\anaconda3\envs\nerr\Lib\site-packages\pyogrio\core.py:26: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


<xarray.Dataset> Size: 174GB
Dimensions:      (time: 26907, lat: 358, lon: 753)
Coordinates:
  * time         (time) datetime64[ns] 215kB 1951-01-01 ... 2024-08-31
  * lat          (lat) float32 1kB 24.56 24.6 24.65 24.69 ... 39.35 39.4 39.44
  * lon          (lon) float32 3kB -106.6 -106.6 -106.5 ... -75.35 -75.31 -75.27
    spatial_ref  int64 8B 0
Data variables:
    tmax         (time, lat, lon) float64 58GB dask.array<chunksize=(20, 358, 753), meta=np.ndarray>
    tmin         (time, lat, lon) float64 58GB dask.array<chunksize=(20, 358, 753), meta=np.ndarray>
    prcp         (time, lat, lon) float64 58GB dask.array<chunksize=(20, 358, 753), meta=np.ndarray>
Attributes: (12/33)
    date_created:               2022-08-27 17:20:14.814647
    date_modified:              2022-08-27 17:20:14.815683
    Conventions:                CF-1.7, ACDD-1.3
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    title:                      nClimGrid-Daily, Gridded Fields
    source:                     GHCN-Daily, nClimGrid-Monthly
    ...                         ...
    time_coverage_start:        1951-01-01
    time_coverage_end:          1951-01-31
    program:                    Center for Weather and Climate/Climate Scienc...
    geospatial_lat_resolution:  0.04167 degrees
    geospatial_lon_resolution:  0.04167 degrees
    platform:                   station

In [9]:
# save metadata separately for later
coords = ds_clip.coords
prcp_attrs = ds_clip.prcp.attrs
tmax_attrs = ds_clip.tmax.attrs
tmin_attrs = ds_clip.tmin.attrs
dims = ds_clip.dims
print(dims)
coords

FrozenMappingWarningOnValuesAccess({'time': 26907, 'lat': 358, 'lon': 753})


Coordinates:
  * time         (time) datetime64[ns] 215kB 1951-01-01 ... 2024-08-31
  * lat          (lat) float32 1kB 24.56 24.6 24.65 24.69 ... 39.35 39.4 39.44
  * lon          (lon) float32 3kB -106.6 -106.6 -106.5 ... -75.35 -75.31 -75.27
    spatial_ref  int64 8B 0

step 10 & 11, round and reduce precision

In [ ]:
# choosing a single time to test data precision, looking for values on order of at least 100
testtime='2000-06-4'
ds_clip.prcp.sel(time=testtime).plot()

In [ ]:
# float16 is probably not enough precision, let's check on a subset of the data

print('loading float16')
prcp_16 = ds_clip.prcp.sel(time=testtime).astype('float16').load()
print('loading float32')
prcp_32 = ds_clip.prcp.sel(time=testtime).astype('float32').load()
print('loading float64')
prcp_64 = ds_clip.prcp.sel(time=testtime).load()

prcp_16.max().item(),prcp_32.max().item(),prcp_64.max().item()

so we can change the data type from float64 to float32 but not go any smaller

In [10]:
# steps 10 & 11
ds = ds_clip.round(decimals=2)
ds = ds_clip.astype('float32')
ds

<xarray.Dataset> Size: 87GB
Dimensions:      (time: 26907, lat: 358, lon: 753)
Coordinates:
  * time         (time) datetime64[ns] 215kB 1951-01-01 ... 2024-08-31
  * lat          (lat) float32 1kB 24.56 24.6 24.65 24.69 ... 39.35 39.4 39.44
  * lon          (lon) float32 3kB -106.6 -106.6 -106.5 ... -75.35 -75.31 -75.27
    spatial_ref  int64 8B 0
Data variables:
    tmax         (time, lat, lon) float32 29GB dask.array<chunksize=(20, 358, 753), meta=np.ndarray>
    tmin         (time, lat, lon) float32 29GB dask.array<chunksize=(20, 358, 753), meta=np.ndarray>
    prcp         (time, lat, lon) float32 29GB dask.array<chunksize=(20, 358, 753), meta=np.ndarray>
Attributes: (12/33)
    date_created:               2022-08-27 17:20:14.814647
    date_modified:              2022-08-27 17:20:14.815683
    Conventions:                CF-1.7, ACDD-1.3
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    title:                      nClimGrid-Daily, Gridded Fields
    source:                     GHCN-Daily, nClimGrid-Monthly
    ...                         ...
    time_coverage_start:        1951-01-01
    time_coverage_end:          1951-01-31
    program:                    Center for Weather and Climate/Climate Scienc...
    geospatial_lat_resolution:  0.04167 degrees
    geospatial_lon_resolution:  0.04167 degrees
    platform:                   station

# write files

I think what we have to do to write each chunk to a separate file is:
- chunk xr arrays in space instead of time
- convert to numpy so we can use to_delayed and ravel
- every worker needs the xr metadata for variable and coordinates
- write a dask delayed function that takes the numpy array data and the metadata separately
- inside the dask delayed function re-create the xarray object and write to file

In [11]:
# chunking along longitude only
nlons=6
ds = ds.chunk({'time':-1,'lat':-1,'lon':nlons})
ds

<xarray.Dataset> Size: 87GB
Dimensions:      (time: 26907, lat: 358, lon: 753)
Coordinates:
  * time         (time) datetime64[ns] 215kB 1951-01-01 ... 2024-08-31
  * lat          (lat) float32 1kB 24.56 24.6 24.65 24.69 ... 39.35 39.4 39.44
  * lon          (lon) float32 3kB -106.6 -106.6 -106.5 ... -75.35 -75.31 -75.27
    spatial_ref  int64 8B 0
Data variables:
    tmax         (time, lat, lon) float32 29GB dask.array<chunksize=(26907, 358, 6), meta=np.ndarray>
    tmin         (time, lat, lon) float32 29GB dask.array<chunksize=(26907, 358, 6), meta=np.ndarray>
    prcp         (time, lat, lon) float32 29GB dask.array<chunksize=(26907, 358, 6), meta=np.ndarray>
Attributes: (12/33)
    date_created:               2022-08-27 17:20:14.814647
    date_modified:              2022-08-27 17:20:14.815683
    Conventions:                CF-1.7, ACDD-1.3
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    title:                      nClimGrid-Daily, Gridded Fields
    source:                     GHCN-Daily, nClimGrid-Monthly
    ...                         ...
    time_coverage_start:        1951-01-01
    time_coverage_end:          1951-01-31
    program:                    Center for Weather and Climate/Climate Scienc...
    geospatial_lat_resolution:  0.04167 degrees
    geospatial_lon_resolution:  0.04167 degrees
    platform:                   station

In [13]:
# a function to return a list of data chunks 
# and the corresponding longitude coord chunks
def xr_ds_to_delayed(ds,varname):
    # chunk the appropriate variable in ds, delay, ravel to list
    var_chunks = ds[varname].data.to_delayed().ravel()
    # xarray doesn't allow chunking of coordinates, so we have to make a new variable to chunk
    # the convoluted process is xarray-->numpy-->xarray-->chunk-->numpy-->delay-->ravel
    lon_chunks = xr.DataArray(ds.lon.data,coords={'lon':('lon',ds.lon.data)}).chunk({'lon':nlons}).data.to_delayed().ravel()
    return var_chunks,lon_chunks

# numpy back to xarray, reattaching metadata and writing chunks to separate files
def write_chunk_to_netcdf(datapath,chunk_id,varname,np_datachunk,xr_time,xr_lat,np_lonchunk,lon_meta,var_meta):
    chunk_id = str(chunk_id).zfill(3)
    # numpy-->xarray
    xr_datachunk = xr.Dataset({varname:(['time','lat','lon'],np_datachunk)},
                              coords={'time':('time',xr_time.data),
                                      'lat':('lat',xr_lat.data),
                                      'lon':('lon',np_lonchunk)})
    # copy over xr metadata
    xr_datachunk.time.attrs=xr_time.attrs
    xr_datachunk.lat.attrs=xr_lat.attrs
    xr_datachunk.lon.attrs=lon_meta
    xr_datachunk[varname].attrs=var_meta

    # clean up metadata
    attrslist=['time','lat','lon',varname]
    for att in attrslist:
        if 'valid_min' in xr_datachunk[att].attrs:
            del xr_datachunk[att].attrs['valid_min']
        if 'valid_max' in xr_datachunk[att].attrs:
            del xr_datachunk[att].attrs['valid_max']
        if (att!=varname) and ('comment' in xr_datachunk[att].attrs):
            del xr_datachunk[att].attrs['comment']
        if 'id' in xr_datachunk[att].attrs:
            del xr_datachunk[att].attrs['id']            
    
    # write file
    xr_datachunk.to_netcdf(datapath+'chunkedlon/'+varname+'_nClimGridDaily_USsouth_'+chunk_id+'.nc')
    return chunk_id

In [14]:
%%time 
var='prcp'
var_chunks,lon_chunks = xr_ds_to_delayed(ds,var)
task_list= [dask.delayed(write_chunk_to_netcdf)(data_dir,id,var,
                                                datachunk,ds.time,ds.lat,
                                                lonchunk,ds.lon.attrs,ds[var].attrs) \
            for id,(datachunk,lonchunk) in enumerate(zip(var_chunks,lon_chunks))]
completed_files = dask.compute(*task_list)
len(completed_files)

CPU times: total: 1min 20s
Wall time: 3min 22s


126

In [15]:
%%time 
var='tmax'
var_chunks,lon_chunks = xr_ds_to_delayed(ds,var)
task_list= [dask.delayed(write_chunk_to_netcdf)(data_dir,id,var,
                                                datachunk,ds.time,ds.lat,
                                                lonchunk,ds.lon.attrs,ds[var].attrs) \
            for id,(datachunk,lonchunk) in enumerate(zip(var_chunks,lon_chunks))]
completed_files = dask.compute(*task_list)
len(completed_files)

CPU times: total: 1min 24s
Wall time: 3min 31s


126

In [ ]:
%%time 
var='tmin'
var_chunks,lon_chunks = xr_ds_to_delayed(ds,var)
task_list= [dask.delayed(write_chunk_to_netcdf)(data_dir,id,var,
                                                datachunk,ds.time,ds.lat,
                                                lonchunk,ds.lon.attrs,ds.prcp.attrs) \
            for id,(datachunk,lonchunk) in enumerate(zip(var_chunks,lon_chunks))]
completed_files = dask.compute(*task_list)

In [ ]:
var='prcp'
files = glob.glob(data_dir+var+'_nClimGridDaily_USsouth_*.nc')
test=xr.open_mfdataset(files)
test

In [ ]:
test[var].isel(time=15).plot()

# old code below to write 1 single file per variable

In [ ]:
%%time
filename='prcp_nClimGridDaily_1951-2024_USsouth.nc'
print('writing',filename)
ds.prcp.to_netcdf(data_dir+filename)

In [ ]:
%%time
filename='tmax_nClimGridDaily_1951-2024_USsouth.nc'
print('writing',filename)
ds.tmax.to_netcdf(data_dir+filename)

In [ ]:
%%time
filename='tmin_nClimGridDaily_1951-2024_USsouth.nc'
print('writing',filename)
ds.tmin.to_netcdf(data_dir+filename)

In [ ]:
test=xr.open_mfdataset(data_dir+filename)
test

In [ ]:
client.shutdown()